In [1]:
%matplotlib inline
#from tensorflow.python.keras.applications.resnet50 import ResNet50
#from tensorflow.python.keras.applications.resnet50 import preprocess_input

from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.python.keras.applications.vgg16 import VGG16

# 画像のリサイズの大きさ（一辺）
img_size = 96

#訓練済みVGGを読込む
vgg= VGG16(include_top=False, input_shape=(img_size,img_size,3))
resnet=vgg

#resnet= ResNet50(include_top=False, input_shape=(img_size,img_size,3))

W0706 16:17:34.904481 25852 deprecation.py:506] From C:\Users\swing\.conda\envs\tensorflow\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [2]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten
from tensorflow.python.keras.models import Model

In [3]:
#訓練済みVGGの最終層に全結合層を追加
top_model = Sequential()
top_model.add(Flatten())
    
model= Model(inputs = resnet.input, outputs = top_model(resnet.output))
for layer in model.layers[:18]:
    layer.trainable = False
    
for i, layer in enumerate(model.layers):
    print(i, layer.name)

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 sequential


In [4]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 96, 96, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 96, 96, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 96, 96, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 48, 48, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 48, 48, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 48, 48, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 24, 24, 128)       0     

In [5]:
import shutil
import os
import numpy as np
from tensorflow.python.keras.preprocessing.image import load_img
from tensorflow.python.keras.preprocessing.image import img_to_array
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix

# 交差検証の分割数（大きい方が良いが実行時間と相談）
n_splits=10

z=[]
label=[]
count=0

#データをすべて読み込む
data_dir_list =['img//exist', 'img//not_exist']
for data_dir_temp in data_dir_list:
    filename = os.listdir(data_dir_temp)
    for i in filename:
        x = load_img(data_dir_temp+'//'+i, target_size=(img_size,img_size))
        x = img_to_array(x)
        x_preproc=(x.copy())/255.
        x_preproc=x_preproc.reshape((img_size,img_size,3))
        z.append(x_preproc)
        label.append(count)
    count+=1

z=np.array(z)
z=model.predict(z).reshape((z.shape[0], -1))
label=np.array(label)    

##### 学習方式を定義する

#PCAとSVMを使用する
#clf = make_pipeline(PCA(n_components=69),SVC(kernel='linear', gamma='auto'))

#PCAを使用しない．SVMのみ
clf = make_pipeline(SVC(kernel='linear', gamma='auto'))

#交差検証の関数を用いてSVMの評価を行う（訓練データとテストデータを自動で入れ替えて実行してくれる．分割数はn_splitsで指定した値）
label_pred = cross_val_predict(clf, z, label, cv=StratifiedKFold(n_splits=n_splits, shuffle=True))

##### 以降結果のまとめ

# Confusion matrixを生成する
conf_matrix = confusion_matrix(label,label_pred)
print('{}分割交差検証の結果'.format(n_splits))
print('識別率:\n {}'.format((conf_matrix[0][0]+conf_matrix[1][1])/np.sum(conf_matrix)))
print('適合率:\n {}'.format(conf_matrix[0][0]/np.sum(conf_matrix[:,0])))
print('感度:\n {}'.format(conf_matrix[0][0]/np.sum(conf_matrix[0,:])))
print('特異度:\n {}'.format(conf_matrix[1][1]/np.sum(conf_matrix[1,:])))
print('confusion matrix:\n{}\n'.format(conf_matrix))

import shutil
shutil.rmtree('tp')
shutil.rmtree('fp')
shutil.rmtree('tn')
shutil.rmtree('fn')
os.mkdir('tp')
os.mkdir('fp')
os.mkdir('tn')
os.mkdir('fn')

count=0
for data_dir_temp in data_dir_list:
    filename = os.listdir(data_dir_temp)
    for i in filename:
        if label[count]==0 and label_pred[count]==0:
            shutil.copy(data_dir_temp+'//'+i,"tp//"+i)
        elif label[count]==1 and label_pred[count]==0:
            shutil.copy(data_dir_temp+'//'+i,"fp//"+i)
        elif label[count]==1 and label_pred[count]==1:
            shutil.copy(data_dir_temp+'//'+i,"tn//"+i)
        elif label[count]==0 and label_pred[count]==1:
            shutil.copy(data_dir_temp+'//'+i,"fn//"+i)
        count+=1

    
##### 未知データに適用するために念のため用意しておきました

print('訓練データをそのままテストデータとしています．適宜変更ください')
# 与えられたデータ全てを学習
clf.fit(z, label)
# zは適当な新しいデータに置き換えてください（アサリ生息マップ作成用データなど）
label_pred = clf.predict(z)

# 確認用
conf_matrix = confusion_matrix(label,label_pred)
print(conf_matrix)
print('識別率:\n {}'.format((conf_matrix[0][0]+conf_matrix[1][1])/np.sum(conf_matrix)))
print('適合率:\n {}'.format(conf_matrix[0][0]/np.sum(conf_matrix[:,0])))
print('感度:\n {}'.format(conf_matrix[0][0]/np.sum(conf_matrix[0,:])))
print('特異度:\n {}'.format(conf_matrix[1][1]/np.sum(conf_matrix[1,:])))


10分割交差検証の結果
識別率:
 0.7120535714285714
適合率:
 0.6752767527675276
感度:
 0.8169642857142857
特異度:
 0.6071428571428571
confusion matrix:
[[366  82]
 [176 272]]

訓練データをそのままテストデータとしています．適宜変更ください
[[426  22]
 [108 340]]
識別率:
 0.8549107142857143
適合率:
 0.797752808988764
感度:
 0.9508928571428571
特異度:
 0.7589285714285714
